In [1]:
import numpy as np 
import tensorflow as tf 
from tensorflow import keras 
from tensorflow.keras.datasets import mnist 
from tensorflow.keras.models import Model 
from tensorflow.keras.layers import Input, Dense, Dropout, Flatten 
from tensorflow.keras.layers import Conv2D, MaxPooling2D, ReLU  
from tensorflow.keras import backend as K 

/home/wyg/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/wyg/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/wyg/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/wyg/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:519: 

In [2]:
# 加载数据
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = np.expand_dims(x_train, axis=3)
x_test = np.expand_dims(x_test, axis=3)

11493376/11490434 [==============================] - 244s 21us/step


In [3]:
# 输入数据标准化
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
# 输入值放缩到0-1之间，归一化
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [4]:
# 使用onehot编码 标签共10个
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

In [5]:
# 建立模型
input_tensor = Input(shape=(28, 28, 1), name='input_tensor')
# 第一层
x = Conv2D(32, (3, 3), name='conv1')(input_tensor)
x = ReLU(name='relu1')(x)
x = MaxPooling2D(pool_size=(2, 2), name='maxpool1')(x)
# 第二层
x = Conv2D(64, (3, 3), name='conv2')(x)
x = ReLU(name='relu2')(x)
x = MaxPooling2D(pool_size=(2, 2), name='maxpool2')(x)
# 第三层
x = Conv2D(128, (3, 3), name='conv3')(x)
x = ReLU(name='relu3')(x)
x = MaxPooling2D(pool_size=(2, 2), name='maxpool3')(x)
# 输出层
x = Flatten(name='flatten')(x)
x = Dense(128)(x)
x = ReLU(name='reluout')(x)

output_tensor = Dense(10, name='output_tensor')(x)

model = Model(inputs=input_tensor, outputs=output_tensor)


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [6]:
# 编译模型：选择 损失函数，优化方法，评价指标
model.compile(loss=keras.losses.CategoricalCrossentropy(),
              optimizer=keras.optimizers.Adam(learning_rate=1e-3),
              metrics=['accuracy'])

In [7]:
# 开始训练
model.fit(x_train, y_train,
          batch_size=128,
          epochs=3,
          verbose=1,
          validation_data=(x_test, y_test))

Train on 60000 samples, validate on 10000 samples
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/3
60000/60000 [==============================] - 58s 967us/sample - loss: 8.1655 - acc: 0.1000 - val_loss: 8.1993 - val_acc: 0.1030
Epoch 2/3
60000/60000 [==============================] - 44s 729us/sample - loss: 8.1619 - acc: 0.0989 - val_loss: 8.1993 - val_acc: 0.1030
Epoch 3/3
60000/60000 [==============================] - 52s 869us/sample - loss: 8.1619 - acc: 0.0989 - val_loss: 8.1993 - val_acc: 0.1030


In [8]:
# 评估模型
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 8.199275217437744
Test accuracy: 0.103


In [9]:
# 存储模型
sess = K.get_session()
saver = tf.train.Saver()
saver.save(sess, './models/float_point/model.ckpt')

'./models/float_point/model.ckpt'